<img src="logoM1.png"  width=70 align=right>
<img src="logoSQM.png" width=65 align=left>

<h1><center> Control estadístico de procesos seis sigma en lotes de productos</center></h1>
<h4><center> Analítica Avanzada - Vicepresidencia Potasio Litio</center></h4>

In [5]:
""" 
File      : lotes_seis_sigma.ipynb
Autor     : Mauricio Bustamante Villarroel
Fecha     : 2020/02/13 (yyyy/mm/dd)
Proposito : Gráfico interactivo en Jupyter para crear gráficos de control estadístico
            de procesos con líneas de reference 6 sigma desde tabla Lims.DETALLE_MUESTRAS_LIMS
            
            - Primera versión de producción que incorpora filtro de análisis/muestras en corrientes
            que no corresponden a lotes pero existen en Lims.
            - Fórmula de ppk está referenciada con el promedio, el cual puede cambiar según requerimientos
            del área de calidad.
            
            -Siguientes pasos: Modificar código para leer datos de especificación de productos desde tabla.
                  
"""

from __future__ import print_function
import matplotlib.pyplot as plt
from math import ceil
import math
import numpy as np
import os 
import pandas as pd
import plotly_express as px
import plotly.graph_objects as go
import pyodbc
from plotly.subplots import make_subplots 

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import warnings


%matplotlib inline
warnings.filterwarnings("ignore")

#DESCOMENTAR SI EN SERVIDO SE TIENE ESTRUCTURA CON UN COMMON_CODE PARA GURDAR FUNCIONES QUE PUEDEN SER
#UTILIZADAS EN MULTIPLES PROYECTOS. POR EL MOMENTO CREADO MANUAL
server = 'sqmstage.database.windows.net'
username = 'stage_usr'
password = 'Sqm.2018'
database = 'ODS_Carbonato'
string_connection = "Driver={ODBC Driver 17 for SQL Server};Server="+ server +";Database="+ database +";uid="+ username +";pwd="+ password +";Integrated Security=False;"
pyodbc_stage = pyodbc.connect(string_connection) 
#project_folder = Path(os.getcwd())
#common_code = project_folder.ancestor(2) + "\\common_code"
#sys.path.append(common_code) #Agregandolo a path para poder importar codigos (funciones o clases) comunes

#Importando codigo comun
#from sqm_database_cnxn import sqm_pyodbc_cnxn

#Creando conexiones pyodbc utilizadas
#pyodbc_stage = sqm_pyodbc_cnxn(database='ODS_Carbonato')


### Leer especificaciones ################################################################
# Dede Excel, que luego será una base de datos en Stage
#project_folder = os.path.abspath('')
#df_esp = pd.read_excel(project_folder + '\\Especificacion_corrientes_v20200221.xlsx')
df_esp = pd.read_sql(
            """ 
             SELECT *  FROM M1.Especificaciones_lotes_SdC """ 
            , pyodbc_stage
            #, index_col='fecha_hora'
            )

#print(df_esp)
df_esp['key_dict'] = [str(c) + ' - ' + d for c, d in zip(df_esp['Corriente'],df_esp['Descripcion']) ]
key_dict_list = df_esp['key_dict'].unique()

dict_selección = {}
tuple_list = []

for i in key_dict_list:
    df_temp = df_esp[ df_esp['key_dict'] == i]
    #Si ambos son no nulos, lista, sino poner maximo, si el maximo es Nan entoces corregir y poner min
    tuple_list = [ (a,u,[str(emin),str(emax)],['min','max']) if math.isnan(emin) == False and math.isnan(emax) == False  
                  else (a,u,str(emax),'max')
                  if math.isnan(emax) == False 
                  else  (a,u,str(emin),'min')
                  for a,u,emin,emax in zip(df_temp['Analisis'],df_temp['Unidad'],df_temp['Esp. Minima'],df_temp['Esp. Maxima'])
                 ]
    dict_selección[i] = tuple_list

lista_corrientes = list(dict_selección.keys())
#print(lista_corrientes)


def get_data(server, username, password, database, query): 
    string_connection = "Driver={ODBC Driver 17 for SQL Server};Server="+ server +";Database="+ database +";uid="+ username +";pwd="+ password +";Integrated Security=False;"      
    cnxn = pyodbc.connect(string_connection)
    sql  = query
    df = pd.read_sql(sql,cnxn)    
    return df


### Definiendo función principal ################################################################
def fx_main(corriente, n_dias_atras, num_std):    
    #Paso 1: Extraer datos de las corrientes a bsucar con todos sus análisis

    df_raw = get_data(
                server = 'clazudbsqmidscdes.database.windows.net' , 
                username = 'cons_datamart',
                password = 'Con$20l9',
                database = 'dataMart_dev',
                query = """             
                select distinct 
                    FECHA_LOGIN    as [Fecha]
                    , FECHA_MUESTRA  as [Fecha de Muestra]
                    , src.NUMERO_MUESTRA as [Número de Muestra]
                    , cast(src.corriente as varchar(10) ) as [Corriente]
                    , concat(src.corriente, '_') as [Número de Corriente]
                    , analisis
                    , unidad_analisis
                    , concat(analisis,'_', unidad_analisis) as [Análisis_Unidad] 
                    , VALOR_ANALISIS as [Valor de Análisis]
                    , PUNTO_MUESTREO as [Punto de Muestreo]
                    , case when VALOR_ANALISIS < 0 then 1 else 0 end as [Valor de Análisis Negativo]
                FROM [Lims].[DETALLE_MUESTRAS_LIMS] as src
                
                inner join [Lims].[Lote_Interno_Pivote] as a
                on src.NUMERO_MUESTRA = a.NUMERO_MUESTRA

                WHERE 
                      a.SQM_LOTE_INTERNO is not null --Tiene que tener un número de lote para eliminar otras muestras
                  and FECHA_MUESTRA >= getdate() -""" + str(n_dias_atras) + 
                """ AND punto_muestreo in ('""" + str(corriente) + """ ')   
                order by FECHA_LOGIN """  
    )

    #display(df_raw)
    #Paso 2: Separar dataframe por cada análisis a graficar
    variables = locals()

    num_graficos = len(dict_selección[corriente])
    #print(num_graficos)
    for c in range(0, int(ceil(len(dict_selección[corriente])/2)) ): #Se necesita N/2 loops (redondeado hacia arriba)
        #Crear indices en pares: (0,1), (2,3), (4,5) ,... etc.
        i = c
        #print("c:",c)
        if c > 0:
            i = c*2
        j = i + 1
        #print("i:",i," j:",j)
        #Crear titulos de variables a graficar
        analisis_unidad  = dict_selección[corriente][i][0] + "_" + dict_selección[corriente][i][1]
        
        dos_limites_ind_1 = isinstance(dict_selección[corriente][i][2], list)
        
        if (dos_limites_ind_1): #Si tiene 2 limites de especificacion (min, max)
            sl1       = float(dict_selección[corriente][i][2][0])
            sl_type1  = dict_selección[corriente][i][3][0]
            sl12      = float(dict_selección[corriente][i][2][1])
            sl_type12 = dict_selección[corriente][i][3][1]
        else:
            sl1      = float(dict_selección[corriente][i][2])
            sl_type1 = dict_selección[corriente][i][3]
        try:
            dos_limites_ind_2 = isinstance(dict_selección[corriente][j][2], list)
            analisis_unidad2 = dict_selección[corriente][j][0] + "_" + dict_selección[corriente][j][1]
            if (dos_limites_ind_2): #Si tiene 2 limites de especificacion (min, max)
                sl2       = float(dict_selección[corriente][j][2][0])
                sl_type2  = dict_selección[corriente][j][3][0]
                sl22      = float(dict_selección[corriente][j][2][1])
                sl_type22 = dict_selección[corriente][j][3][1]
            else:
                sl2      = float(dict_selección[corriente][j][2])
                sl_type2 = dict_selección[corriente][j][3]
        except IndexError:
            analisis_unidad2 = ''
            sl2 = None
            sl_type2 = None
            sl22 = None
            sl_type22 = None
        

        #Filtrar datos para cada gráfico
        #variables["df{0}".format(i)] = df_raw.loc[(df_raw['Punto de Muestreo']==corriente) & (df_raw['Análisis_Unidad'] == analisis_unidad)]
        df1 = df_raw.loc[(df_raw['Punto de Muestreo']==corriente) & (df_raw['Análisis_Unidad'] == analisis_unidad)]
        df2 = df_raw.loc[(df_raw['Punto de Muestreo']==corriente) & (df_raw['Análisis_Unidad'] == analisis_unidad2)]

        df1_neg = df1[df1['Valor de Análisis Negativo'] == 1]
        df2_neg = df2[df2['Valor de Análisis Negativo'] == 1]

        #display(df1)
        #display(df2)
        
        if df1.empty == True:
            punto_muestreo = ''
            count = None
            count_neg = None
            mean_value = None
            min_value = None
            max_value = None
            std_value = None
            uper_lim  = None
            lower_lim = None
            y_max = None
            y_min = None
            lcs_label = "Sin datos"
            lci_label = ""
            ppk1 = ""
            ppk1_class = ""
            sl_type1 = ""

        else:
            punto_muestreo = str(df1['Punto de Muestreo'].unique())[2:-2]
            count = df1['Valor de Análisis'].count()
            count_neg  = df1['Valor de Análisis Negativo'].sum()
            mean_value = df1['Valor de Análisis'].mean().round(4)
            min_value  = df1['Valor de Análisis'].min().round(4)
            max_value  = df1['Valor de Análisis'].max().round(4)
            std_value  = df1['Valor de Análisis'].std().round(4)
            uper_lim   = (mean_value + num_std*std_value).round(4)
            lower_lim  = (mean_value - num_std*std_value).round(4)
            y_max = mean_value + 5*std_value
            y_min = mean_value - 5*std_value
            lcs_label = "LCS: Media + " + str(num_std) +"Desv Std"
            lci_label = "LCI: Media - " + str(num_std) +"Desv Std"
            ppk1_class = ""
            
            if (dos_limites_ind_1): #Si tiene 2 limites
                if sl_type1  == 'min': 
                    pp1_min  = (mean_value - sl1)/(num_std*std_value)
                if sl_type1  == 'max': 
                    pp1_max  = (sl1 - mean_value)/(num_std*std_value)
       
                if sl_type12 == 'min': 
                    pp1_min  = (mean_value - sl1)/(num_std*std_value)
                if sl_type12 == 'max': 
                    pp1_max  = (sl1 - mean_value)/(num_std*std_value)    
            
                ppk1 = (min(pp1_min, pp1_max)).round(2) 
                
                if ppk1 < 1:
                    ppk1_class = 'Proceso caótico' 
                if ppk1 >= 1 and ppk1 < 1.33:
                    ppk1_class = 'Capacidad 3 Sigma' 
                if ppk1 >= 1.33 and ppk1<1.67:
                    ppk1_class = 'Capacidad 4 Sigma' 
                if ppk1 >= 1.67 and ppk1 < 2:
                    ppk1_class = 'Capacidad 4 Sigma' 
                if ppk1 >= 2:
                    ppk1_class = 'Capacidad 6 Sigma' 
      
            else: #Si es solo un valor
                if sl_type1 == 'min': 
                    pp1 = (mean_value - sl1)/(3*std_value)
                if sl_type1 == 'max': 
                    pp1 = (sl1 - mean_value)/(3*std_value)

                ppk1 = (pp1).round(2)
                if ppk1 < 1:
                    ppk1_class = 'Proceso caótico' 
                if ppk1 >= 1 and ppk1 < 1.33:
                    ppk1_class = 'Capacidad 3 Sigma' 
                if ppk1 >= 1.33 and ppk1<1.67:
                    ppk1_class = 'Capacidad 4 Sigma' 
                if ppk1 >= 1.67 and ppk1<2:
                    ppk1_class = 'Capacidad 4 Sigma' 
                if ppk1 >= 2:
                    ppk1_class = 'Capacidad 6 Sigma' 


        dtmin, dtmax = df1['Fecha'].min(), df1['Fecha'].max()

        #Creando gráfico 
        fig_main = make_subplots( 
            rows=1 , cols=4,
            column_widths=[0.4, 0.15, 0.4, 0.15],
            specs=[ [{"type":"scatter"}, {"type":"table"}, {"type":"scatter"}, {"type":"table"} ] ],
            subplot_titles=(
                'Gráfico '+ analisis_unidad, 
                'Estadísticas '+ analisis_unidad, 
                'Gráfico '+analisis_unidad2, 
                'Estadísticas '+ analisis_unidad2
            )
        )
   

        ## Creando gráfico de lineas seis sigma DF1 ###################################################################
        fig_main.add_trace(go.Scatter(x = [dtmin, dtmax, dtmax, dtmin],
                                   y = [lower_lim, lower_lim, uper_lim, uper_lim], 
                                   mode='lines',
                                   fill='toself', 
                                   line_color='#98FB98',
                                   opacity=0.5,
                                   showlegend=False,
                                   hoverinfo='none')
                           , row=1, col = 1
                       )
    
        fig_main.add_trace(
            go.Scatter(
                x=df1['Fecha'], 
                y =df1['Valor de Análisis'], 
                text = df1['Número de Muestra'],
                name="Datos" ,
                line_color='black',
                hovertemplate = "Fecha: %{x} <br>Valor: %{y} <br>N° Muestra: %{text}"
            ) , row=1, col = 1
        )

        fig_main.add_trace(
            go.Scatter(
                x=df1_neg['Fecha'], 
                y =df1_neg['Valor de Análisis'], 
                text = df1['Número de Muestra'],
                name="Datos Negativos" ,
                line_color='red',
                hovertemplate = "Fecha: %{x} <br>Valor: %{y} <br>N° Muestra: %{text}"
            ) , row=1, col = 1
        )

        fig_main.add_trace(go.Scatter(x=df1['Fecha'], y =[uper_lim]*len(df1) , name="LCS", mode='lines', line_color='blue', line_width=2)
                          , row=1, col = 1)
        fig_main.add_trace(go.Scatter(x=df1['Fecha'], y =[lower_lim]*len(df1), name="LCI", mode='lines', line_color='blue', line_width=2)
                          , row=1, col = 1)
        fig_main.add_trace(go.Scatter(x=df1['Fecha'], y =[mean_value]*len(df1),name="Media",mode='lines',line_color='green',line_width=1)
                          , row=1, col = 1)
        #Limite de especificacion LLEE
        if (dos_limites_ind_1):
            fig_main.add_trace(go.Scatter(x=df1['Fecha'], y =[sl1]*len(df1),name="Especif. "+ sl_type1,mode='lines',line_color='red',line_width=2)
                          , row=1, col = 1)
            fig_main.add_trace(go.Scatter(x=df2['Fecha'], y =[sl12]*len(df1),name="Especif. " + str(sl_type12),mode='lines',line_color='red',line_width=2)
                          , row=1, col = 1)
        else:
            fig_main.add_trace(go.Scatter(x=df1['Fecha'], y =[sl1]*len(df1),name="Especif. "+ sl_type1,mode='lines',line_color='red',line_width=2)
                          , row=1, col = 1)
            
        #Tabla con estadísticos
        if (dos_limites_ind_1):
            especif_col_name = 'Especif. ' + sl_type1 + '/' + sl_type12
            sl1_label = str(sl1) + '/' + str(sl12)
        else:
            especif_col_name = 'Especif. ' + sl_type1
            sl1_label = sl1
            
        fig_main.add_trace(
            go.Table(
                header=dict(
                    values=["Estadístico", "Valor"],
                    font=dict(size=10),
                    align="left"
                ),
                cells=dict(
                    values= [
                            ['N° Datos', 'N° Datos negativos','Mínimo' , 'Promedio', 'Máximo', 'Desviación Estándar', 
                             especif_col_name, 'LCS', 'LCI', 'PPK'] , 
                            [count, count_neg, min_value, mean_value, max_value, std_value, 
                             sl1_label, uper_lim, lower_lim, str(ppk1) + ': ' + ppk1_class ]
                        ],
                    align = "left")
                ), row = 1, col=2
        )
 
        
        ## Creando gráfico de lineas seis sigma DF2 ###################################################################            
        if df2.empty == True:
            punto_muestreo = ''
            count = None
            count_neg = None
            mean_value = None
            min_value = None
            max_value = None
            std_value = None
            uper_lim  = None
            lower_lim = None
            y_max = None
            y_min = None
            lcs_label = "Sin datos"
            lci_label = ""
            ppk2 = ""
            ppk2_class = ""
            sl_type2 = ""
            
        else:
            punto_muestreo = str(df2['Punto de Muestreo'].unique())[2:-2]
            count = df2['Valor de Análisis'].count()
            count_neg = df2['Valor de Análisis Negativo'].sum()
            mean_value = df2['Valor de Análisis'].mean().round(4)
            min_value  = df2['Valor de Análisis'].min().round(4)
            max_value  = df2['Valor de Análisis'].max().round(4)
            std_value  = df2['Valor de Análisis'].std().round(4)
            uper_lim   = (mean_value + num_std*std_value).round(4)
            lower_lim  = (mean_value - num_std*std_value).round(4)
            y_max = mean_value + 5*std_value
            y_min = mean_value - 5*std_value
            lcs_label = "LCS: Media + " + str(num_std) +"Desv Std"
            lci_label = "LCI: Media - " + str(num_std) +"Desv Std"
            ppk2_class = ""
            
            if (dos_limites_ind_2): #Si tiene 2 limites
                if sl_type2  == 'min': 
                    pp2_min  = (mean_value - sl2)/(num_std*std_value)
                if sl_type2  == 'max': 
                    pp2_max  = (sl2 - mean_value)/(num_std*std_value)
       
                if sl_type22 == 'min': 
                    pp2_min  = (mean_value - sl2)/(num_std*std_value)
                if sl_type22 == 'max': 
                    pp2_max  = (sl2 - mean_value)/(num_std*std_value)  
 
                ppk2 = (min(pp2_min, pp2_max)).round(2) 
                if ppk2 < 1:
                    ppk2_class = 'Proceso caótico' 
                if ppk2 >= 1 and ppk2 < 1.33:
                    ppk2_class = 'Capacidad 3 Sigma' 
                if ppk2 >= 1.33 and ppk2 < 1.67:
                    ppk2_class = 'Capacidad 4 Sigma' 
                if ppk2 >= 1.67 and ppk2 < 2:
                    ppk2_class = 'Capacidad 4 Sigma' 
                if ppk2 >= 2:
                    ppk2_class = 'Capacidad 6 Sigma' 
                    
            else:        
                if sl_type2 == 'min': 
                    pp2 = (mean_value - sl2)/(3*std_value)
                if sl_type2 == 'max': 
                    pp2 = (sl2 - mean_value)/(3*std_value)

                ppk2 = (pp2).round(2)
                if ppk2 < 1:
                    ppk2_class = 'Proceso caótico' 
                if ppk2 >= 1 and ppk2 < 1.33:
                    ppk2_class = 'Capacidad 3 Sigma' 
                if ppk2 >= 1.33 and ppk2 < 1.67:
                    ppk2_class = 'Capacidad 4 Sigma' 
                if ppk2 >= 1.67 and ppk2 < 2:
                    ppk2_class = 'Capacidad 4 Sigma' 
                if ppk2 >= 2:
                    ppk2_class = 'Capacidad 6 Sigma' 
                

        dtmin, dtmax = df2['Fecha'].min(), df2['Fecha'].max()
        
        fig_main.add_trace(go.Scatter(x = [dtmin, dtmax, dtmax, dtmin],
                                   y = [lower_lim, lower_lim, uper_lim, uper_lim], 
                                   mode='lines',
                                   fill='toself', 
                                   line_color='#98FB98',
                                   opacity=0.5,
                                   showlegend=False,
                                   hoverinfo='none')
                           , row=1, col = 3
                       )

        fig_main.add_trace(
            go.Scatter(
                x=df2['Fecha'], 
                y =df2['Valor de Análisis'], 
                text = df2['Número de Muestra'],
                name="Datos" ,
                line_color='black',
                hovertemplate = "Fecha: %{x} <br>Valor: %{y} <br>N° Muestra: %{text}"
            ) , row=1, col = 3
        )

        fig_main.add_trace(
            go.Scatter(
                x=df2_neg['Fecha'], 
                y =df2_neg['Valor de Análisis'], 
                text = df2['Número de Muestra'],
                name="Datos Negativos" ,
                line_color='red',
                hovertemplate = "Fecha: %{x} <br>Valor: %{y} <br>N° Muestra: %{text}"
            ) , row=1, col = 3
        )

        fig_main.add_trace(go.Scatter(x=df2['Fecha'], y =[uper_lim]*len(df2) , name="LCS", mode='lines', line_color='blue', line_width=2)
                          , row=1, col = 3)
        fig_main.add_trace(go.Scatter(x=df2['Fecha'], y =[lower_lim]*len(df2), name="LCI", mode='lines', line_color='blue', line_width=2)
                          , row=1, col = 3)
        fig_main.add_trace(go.Scatter(x=df2['Fecha'], y =[mean_value]*len(df2),name="Media",mode='lines',line_color='green',line_width=1)
                          , row=1, col = 3)   
        #Limite de especificacion LLEE
        if (dos_limites_ind_2):
            fig_main.add_trace(go.Scatter(x=df2['Fecha'], y =[sl2]*len(df2),name="Especif. " + str(sl_type2),mode='lines',line_color='red',line_width=2)
                          , row=1, col = 3)
            fig_main.add_trace(go.Scatter(x=df2['Fecha'], y =[sl22]*len(df2),name="Especif. " + str(sl_type22),mode='lines',line_color='red',line_width=2)
                          , row=1, col = 3)
        else:
            fig_main.add_trace(go.Scatter(x=df2['Fecha'], y =[sl2]*len(df2),name="Especif. " + str(sl_type2),mode='lines',line_color='red',line_width=2)
                          , row=1, col = 3)
    
        #Tabla con estadísticos
        if (dos_limites_ind_2):
            especif_col_name = 'Especif. ' + sl_type2 + '/' + sl_type22
            sl2_label = str(sl2) + '/' + str(sl22)
        else:
            especif_col_name = 'Especif. ' + sl_type2
            sl2_label = sl2
             
        fig_main.add_trace(
            go.Table(
                header=dict(
                    values=["Estadístico", "Valor"],
                    font=dict(size=10),
                    align="left"
                ),
                cells=dict(
                    values= [
                            ['N° Datos', 'N° Datos negativos','Mínimo' , 'Promedio', 'Máximo', 'Desviación Estándar', 
                             especif_col_name, 'LCS', 'LCI', 'PPK'] , 
                            [count, count_neg, min_value, mean_value, max_value, std_value, 
                             sl2_label, uper_lim, lower_lim, str(ppk2) + ': ' + ppk2_class ]
                        ],
                    align = "left")
                ), row = 1, col=4
        )
        
        fig_main.update_layout(showlegend=False ) #, title='Corriente ' + corriente )
        
        fig_main.show()
    

    return None

### Ejecutar Jupyter Widget interactivo ################################################################

interact_manual(fx_main,
    corriente= widgets.Dropdown(
                            options= lista_corrientes,
                            description='Corriente:',
                            disabled=False,
                        ), 
    n_dias_atras = widgets.IntSlider(
                            min=7,
                            max=120, 
                            step=1, 
                            value=60, 
                            description='Días atrás:',
                            disabled=False,
                            continuous_update=False,
                            orientation='horizontal',
                            readout=True,
                            readout_format='d'
                        ),
    num_std =  widgets.IntSlider(min=1, max=3, step=1, value=3, description='Num desv std'),
)
# Subir excel a base de datos desde una interface Dash.
#Otra opción es cargarlo desde Jupyter y no subir a BD y transformarlo a DF
#Dejar una opción pro default, y si se carga algo, actualizarlo.


interactive(children=(Dropdown(description='Corriente:', options=('901 - CRY9000.00', '902 - GRA9000.00', '906…

<function __main__.fx_main(corriente, n_dias_atras, num_std)>